# **Face Recognition for Attendance System**

This Python script uses OpenCV and the `face_recognition` library to create an automated face recognition-based attendance system. The program identifies the employees from a dataset of images and marks their attendance in a CSV file using live webcam footage.

### **Dependencies**
```bash
pip install opencv-python face-recognition numpy


In [4]:
import cv2
import face_recognition
import os
import numpy as np
from datetime import datetime
import time 
import pickle

- **cv2**: OpenCV library for handling image and video processing.
- **face_recognition**: Library to handle face detection and recognition.
- **os**: Module to interact with the file system (for loading datasets).
- **numpy**: Library for array manipulation.
- **datetime**: To record the time and date when the attendance is marked.
- **time**: Used to manage time-related functions.
- **pickle**: Python module to serialize and deserialize objects (not used in this snippet).


### **1.Load the datset**

In [5]:
path = 'dataset/'

In [6]:
# Initialize lists to store images and class names (employee names)
images = []
classNames = []

# Get the list of folders in the dataset directory
folders = os.listdir(path)

# Loop through each folder (representing an employee) in the dataset directory
for folder in folders:
    folder_path = os.path.join(path, folder)
    
    # Check if it's a directory (i.e., a folder for an employee)
    if os.path.isdir(folder_path):
        # List all images inside the employee's folder
        for img_name in os.listdir(folder_path):
            img_path = os.path.join(folder_path, img_name)
            curImg = cv2.imread(img_path)
            
            # Append the image and the employee's name (folder name)
            if curImg is not None:
                images.append(curImg)
                classNames.append(folder)


- **path**: Directory containing folders where each folder corresponds to a person (employee).
- **images**: List to store images from the dataset.
- **classNames**: List to store the employee names, derived from folder names.
The code loops through each folder (employee) and each image inside that folder, storing them in the respective lists.

### **2.Encode Faces**

In [7]:
# Function to encode the faces from the images
def findEncodings(images):
    encodeList = []
    for img in images:
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        encoded_face = face_recognition.face_encodings(img)[0]
        encodeList.append(encoded_face)
    return encodeList
encoded_face_train = findEncodings(images)

- **findEncodings()**: Function to convert each image to RGB (required for face_recognition) and encode it using the face_recognition.face_encodings() function. The result is a list of face encodings that will be used to compare faces during live detection.
- **encoded_face_train**: List of encoded faces from the dataset

### **3.Mark Attendance**

In [8]:
def markAttendance(name):
    with open('Attendance.csv','r+') as f:
        myDataList = f.readlines()
        nameList = []
        for line in myDataList:
            entry = line.split(',')
            nameList.append(entry[0])
        if name not in nameList:
            now = datetime.now()
            time = now.strftime('%I:%M:%S:%p')
            date = now.strftime('%d-%B-%Y')
            f.writelines(f'n{name}, {time}, {date}')

- **markAttendance()**: This function checks the attendance file Attendance.csv. If the employee’s name is not already in the file, it appends their name, along with the current time and date, indicating attendance has been marked.

### **4.Real-Time Face Recognition and Attendance**

In [12]:

# Start video capture from the default webcam
cap = cv2.VideoCapture(0)

# Initialize a flag to indicate whether attendance has been marked
attendance_done = False

while True:
    success, img = cap.read()
    imgS = cv2.resize(img, (0, 0), None, 0.25, 0.25)  # Scale down the frame
    imgS = cv2.cvtColor(imgS, cv2.COLOR_BGR2RGB)  # Convert the frame to RGB format

    # Detect face locations and encode the faces in the frame
    faces_in_frame = face_recognition.face_locations(imgS)
    encoded_faces = face_recognition.face_encodings(imgS, faces_in_frame)

    # Loop through each face found in the frame
    for encode_face, faceloc in zip(encoded_faces, faces_in_frame):
        matches = face_recognition.compare_faces(encoded_face_train, encode_face)
        faceDist = face_recognition.face_distance(encoded_face_train, encode_face)

        # Find the closest match (smallest distance)
        matchIndex = np.argmin(faceDist)

        if matches[matchIndex]:
            name = classNames[matchIndex].capitalize()  # Capitalize the name

            # Get the face location (scaled back to original size)
            y1, x2, y2, x1 = faceloc
            y1, x2, y2, x1 = y1 * 4, x2 * 4, y2 * 4, x1 * 4

            # Draw a rectangle around the detected face
            cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 2)
            
            # Draw a filled rectangle for the label below the face
            cv2.rectangle(img, (x1, y2 - 35), (x2, y2), (0, 255, 0), cv2.FILLED)
            
            # Add the name of the recognized employee
            cv2.putText(img, name, (x1 + 6, y2 - 5), cv2.FONT_HERSHEY_COMPLEX, 1, (255, 255, 255), 2)

            # Mark the employee's attendance
            if not attendance_done:
                markAttendance(name)
                attendance_done = True  # Set the flag to True once attendance is marked
                print(f"Attendance marked for {name}")

    # Display the webcam feed
    cv2.imshow('webcam', img)

    # Break the loop if attendance is done or 'q' is pressed
    if attendance_done or cv2.waitKey(1) & 0xFF == ord('q'):
        break

# Release the webcam and close OpenCV windows
cap.release()
cv2.destroyAllWindows()


Attendance marked for Haris


- **cap = cv2.VideoCapture(0)**: Opens the default webcam.
- **Face Detection**: Each frame from the webcam is resized and converted to RGB. face_recognition.face_locations() detects faces, and face_recognition.face_encodings() encodes them.
- **Face Matching**: For each detected face, it compares with the trained encodings using face_recognition.compare_faces() and measures the distance to find the closest match.
- **Drawing Bounding Box**: A rectangle is drawn around the recognized face, and the employee’s name is displayed.
- **Attendance Marking**: Once a face is recognized and attendance is marked, the process stops.

### **5: Exit and Clean Up**
The webcam feed is closed, and all OpenCV windows are destroyed once the process is complete.